In [ ]:

!pip install flask-ngrok  # For running Flask server in Colab

# Import required libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import sqlite3

# Sample data for books
books_data = pd.DataFrame({
    'BookID': [1, 2, 3, 4],
    'Title': ['Book A', 'Book B', 'Book C', 'Book D'],
    'Genre': ['Fiction', 'Mystery', 'Science Fiction', 'Romance']
})

# Sample data for users
users_data = pd.DataFrame({
    'UserID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Email': ['alice@example.com', 'bob@example.com', 'charlie@example.com']
})

# Sample data for transactions
transactions_data = pd.DataFrame({
    'TransactionID': [1, 2, 3],
    'UserID': [1, 2, 3],
    'BookID': [1, 2, 3],
    'BorrowDate': ['2024-09-01', '2024-09-02', '2024-09-03'],
    'ReturnDate': [None, '2024-09-10', None]
})

print("Books Data:\n", books_data)
print("\nUsers Data:\n", users_data)
print("\nTransactions Data:\n", transactions_data)


# Example: Create a simple recommendation system based on genre similarity

# Convert genres to TF-IDF features
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books_data['Genre'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get book recommendations
def get_recommendations(book_title, cosine_sim=cosine_sim):
    idx = books_data[books_data['Title'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:3]  # Get top 2 similar books
    book_indices = [i[0] for i in sim_scores]
    return books_data['Title'].iloc[book_indices]

# Example usage
recommended_books = get_recommendations('Book A')
print("Recommended Books: ", recommended_books.tolist())


# Set up SQLite database
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    BookID INTEGER PRIMARY KEY,
    Title TEXT,
    Genre TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    UserID INTEGER PRIMARY KEY,
    Name TEXT,
    Email TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    TransactionID INTEGER PRIMARY KEY,
    UserID INTEGER,
    BookID INTEGER,
    BorrowDate DATE,
    ReturnDate DATE
)
''')

conn.commit()
print("Database and tables created successfully!")


# Example: Simple user clustering based on borrowing history
interaction_matrix = pd.DataFrame([
    [1, 5, 0, 0, 4],  # User 1
    [2, 4, 1, 0, 3],  # User 2
    [3, 0, 0, 5, 0],  # User 3
], columns=['UserID', 'BookA', 'BookB', 'BookC', 'BookD'])

# Apply K-Means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
interaction_matrix['Cluster'] = kmeans.fit_predict(interaction_matrix.drop('UserID', axis=1))

print("User Clusters:\n", interaction_matrix)


from flask import Flask, jsonify

app = Flask(__name__)



# Simple API to return recommendations
@app.route('/recommendations/<book_title>', methods=['GET'])
def recommend_books(book_title):
    recommended_books = get_recommendations(book_title)
    return jsonify({'recommended_books': recommended_books.tolist()})

if __name__ == '__main__':
    app.run()



Books Data:
    BookID   Title            Genre
0       1  Book A          Fiction
1       2  Book B          Mystery
2       3  Book C  Science Fiction
3       4  Book D          Romance

Users Data:
    UserID     Name                Email
0       1    Alice    alice@example.com
1       2      Bob      bob@example.com
2       3  Charlie  charlie@example.com

Transactions Data:
    TransactionID  UserID  BookID  BorrowDate  ReturnDate
0              1       1       1  2024-09-01        None
1              2       2       2  2024-09-02  2024-09-10
2              3       3       3  2024-09-03        None
Recommended Books:  ['Book C', 'Book B']
Database and tables created successfully!
User Clusters:
    UserID  BookA  BookB  BookC  BookD  Cluster
0       1      5      0      0      4        0
1       2      4      1      0      3        0
2       3      0      0      5      0        1
 * Serving Flask app '__main__'
 * Debug mode: off


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
